# Повторяем питон 

## 0. Импорты

In [1]:
import pandas as pd
import scipy as sp
import numpy as np
import random
import os
import json
import sys
from tqdm import tqdm
import random

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2

from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_array

In [2]:
morph = pymorphy2.MorphAnalyzer()
stops = stopwords.words('russian')
# nltk.download('punkt')

## 1. Про пути

- Абсолютные пути - это очень плохая идея в любом проекте серьезнее домашки первого курса. Причина: скорее всего кто-то (преподаватель, ассистент, коллега, ученик) будет пытаться запускать ваш код, и будет очень расстроен, если ему придется лезть вглубь каких-то непонятных ему функций просто для того, чтобы прпавильно прочитать файл
</br>Например, из папки `/Users/some_user/study/something` лучше обозначить путь так: `/some_dir`, чем так: `/Users/some_user/study/something/some_dir`
- Старайтесь собирать пути через `os.path.join` (или любую другую либу), так как он умеет внутри себя учитывать особенности типа наклона слэша (как вы знаете, он разный у unix-based систем и windows)
- Вообще, библиотека `os` сейчас уже скорее deprecated, все стараются пользоваться `pathlib`, но для каких-то простых штук старая библиотека все еще удобнее, так как намного проще. Но использование красивых решений поощряется :)

## 2. Нотация

In [3]:
def do_something(lst: list[str], bad: bool = False) -> list[str]:
    '''
    This function does something (bad or good) to list
    :param lst: list of input strings
    :param bad: marker if bad action should be done
    :return: sorted (or not...) input list
    '''
    if bad:
        random.shuffle(lst)
        return lst
    else:
        return sorted(lst)

In [4]:
lst = ['python', 'anaconda', 'spider', 'zoo']

In [5]:
do_something(lst, bad=False)

['anaconda', 'python', 'spider', 'zoo']

In [6]:
do_something(lst, bad=True)

['zoo', 'spider', 'anaconda', 'python']

## 3. Ссылки & Co

Питон хранит все данные по ссылке, иногда это создает проблемы в неожиданных местах. `id()` - функция, которая позволяет получить индивидуальный номер объекта (а в питоне все объекты), то есть узнать, как питон ссылается на него в памяти

In [7]:
a = 0
b = 1
a_list = [0, b]
id(0), id(a), id(a_list[0])

(4333748432, 4333748432, 4333748432)

In [8]:
id(1), id(b), id(a_list[1])

(4333748464, 4333748464, 4333748464)

In [9]:
с = 2
id(2), id(с)

(4333748496, 4333748496)

А вот с более сложными типами будет интереснее. Какой ответ ожидается в ячейке ниже?

In [10]:
test_lst1 = [0, 4, 7, 9]
test_lst2 = [2, 6, 3, 8]
test_lst3 = [0, 4, 7, 9]
id(test_lst1), id(test_lst2), id(test_lst3)

(10870542400, 10870550784, 10870194304)

А теперь?

In [11]:
test_lst4 = test_lst1
id(test_lst1), id(test_lst3), id(test_lst4)

(10870542400, 10870194304, 10870542400)

Еще есть прикольная функция `sys.getrefcount`. Я слабо себе представляю, как она может пригодиться вам в жизни (если вы не занимаетесь хардкодом, но тогда вы скорее всего делаете это не на питоне...), но понять хранение питона точно поможет. Надо заметить, что эта функция внутри себя создает еще одну ссылку (возможно, для очень сложным структур больше, но тут я не уверена), так что полученное значение надо уменьшать на единицу

In [12]:
sys.getrefcount([])

1

In [13]:
sys.getrefcount(test_lst2)

2

In [14]:
sys.getrefcount(test_lst1)

3

In [16]:
sys.getrefcount(test_lst4)

3

А вот ниже мы, кажется, случайно увидели что-то очень для питона приватное: мы явно не ссылаемся на нули в таком количестве, значит, это информация о ссылках во внутренней струкутре питоне

In [17]:
sys.getrefcount(0), sys.getrefcount(True), sys.getrefcount(None)

(14065, 11733, 96565)

Итак, с более сложными струкутрами данных питон начинает мухлевать и экономить усилия, дублируя в памяти ссылки, а не сами объекты. Это может ускорить работу и сэкономить память, но провоцирует опасные ситуации. Не баг, а фича, но фича с подвохом. 

Теперь о подвохах

In [18]:
test_lst1, test_lst3, test_lst4

([0, 4, 7, 9], [0, 4, 7, 9], [0, 4, 7, 9])

In [19]:
test_lst1.append(10)

Что мы ожидаем увидеть?

In [20]:
test_lst1, test_lst3, test_lst4

([0, 4, 7, 9, 10], [0, 4, 7, 9], [0, 4, 7, 9, 10])

Теперь о менее очевидном, но более опасном
</br>
Пусть надо написать функцию, которая считает сумму всех элементов в двух списках. Узнав о классной встроенной функции `sum()` мы решили, что просто сложим все элемерты в первый список и посчитаем сумму. Тут со стороны кажется, что сплошные плюсы: мы экономим строки кода, экономим место в переменных...

In [21]:
def test_func1(lst1, lst2):
    lst1 += lst2
    return sum(lst1)

In [22]:
test_func1(test_lst1, test_lst2)

49

Что будет выведено дальше?

In [23]:
test_lst1, test_lst2

([0, 4, 7, 9, 10, 2, 6, 3, 8], [2, 6, 3, 8])

А тепреь еще интереснее

In [24]:
test_lst4

[0, 4, 7, 9, 10, 2, 6, 3, 8]

Казалось бы, и в первом, и во втором случае мы его вообще не трогали, во втором мы еще и не сохраняли никуда резульатат, но питон все решил за нас. Если кому-то хочется пример из жизни, то у меня есть "увлекательная" история о том, как мне это усложнило работу над курсовой :)

Еще один пример, теперь про значения по умолчанию. Пусть нам надо написать функцию, которая получает на вход список пассажиров автобуса, имя пассажира и маркер того, что надо сделать: высадить или посадить. По умолчанию автобус пустой

In [25]:
def bus_func(bus=[], name=None, take=True):
    if name is not None:
        if take:
            bus.append(name)
        else:
            if name in bus:
                bus.remove(name) # высаживаем первого пассажира с таким именем... кому-то не повезет :)
    return bus

Проверяем, что все работает

In [26]:
my_bus = []
my_bus = bus_func(my_bus, 'Kate', True)
my_bus

['Kate']

In [27]:
my_bus = bus_func(my_bus, 'Max', True)
my_bus = bus_func(my_bus, 'Kate', False)
my_bus

['Max']

Посадим в пустой автобус кого-нибудь еще

In [28]:
my_bus2 = bus_func(name='Alice', take=True)
my_bus2 = bus_func(my_bus2, 'Peter', True)
my_bus2

['Alice', 'Peter']

Теперь в другой пустой автобус

In [29]:
my_bus3 = bus_func(name='John', take=True)
my_bus3

['Alice', 'Peter', 'John']

И сделаем просто пустой автобус. Вдруг пригодится

In [30]:
my_bus4 = bus_func()
my_bus4

['Alice', 'Peter', 'John']

Вспомним про второй автобус и высадим кого-нибудь оттуда

In [31]:
my_bus2 = bus_func(my_bus3, 'Peter', False)
my_bus2

['Alice', 'John']

А теперь посмотрим, что же произошло с функцией. У каждой функции есть атрибут `__defaults__`, в котором содержаться значения по умолчанию аргументов этой функции

In [32]:
bus_func.__defaults__

(['Alice', 'John'], None, True)

Ну, теперь это автобус, у которого есть пассажиры по умолчанию :)

Теперь разберемся, почему так происходит.

In [33]:
class MyList(list):
    def __new__(cls, *args, **kwargs):
        print('hello, im here')
        return super(MyList, cls).__new__(cls, *args, **kwargs)
    
    def __init__(self, *args, **kwargs):
        print('and here')
        super().__init__(*args, **kwargs)

In [34]:
def bus_func_cust(bus=MyList(), name=None, take=True):
    if name is not None:
        if take:
            bus.append(name)
        else:
            if name in bus:
                bus.remove(name)
    return bus

hello, im here
and here


In [35]:
my_bus2 = bus_func_cust(name='Alice', take=True)
my_bus2 = bus_func_cust(my_bus2, 'Peter', True)
my_bus2

['Alice', 'Peter']

In [36]:
my_bus4 = bus_func_cust()
my_bus4

['Alice', 'Peter']

In [37]:
bus_func_cust.__defaults__

(['Alice', 'Peter'], None, True)

## 4. Сравнения

Теперь с темы ссылок перейдем к сравнениям, они связаны. В питоне есть два способа проверить равенство двух объектов: </br>`a == b`</br>`a is b` </br>Давайте вернем наши чудесные списки

In [38]:
test_lst1 = [0, 4, 7, 9]
test_lst2 = [2, 6, 3, 8]
test_lst3 = [0, 4, 7, 9]
test_lst4 = test_lst1

Результаты какой пары из трех ячеек ниже совпадут?

In [39]:
test_lst1 == test_lst2, test_lst1 == test_lst3, test_lst1 == test_lst4

(False, True, True)

In [40]:
test_lst1 is test_lst2, test_lst1 is test_lst3, test_lst1 is test_lst4

(False, False, True)

In [41]:
id(test_lst1) == id(test_lst2), id(test_lst1) == id(test_lst3), id(test_lst1) == id(test_lst4)

(False, False, True)

Данные операторы отличаются тем, что один сравнивает содерждание объектов, а другой - их идентификаторы. Поэтому в большинстве случаев надо использовать `==`: нас чаще интересует содержание, а не странные питоновские нюансы питона. Однако сравнение с уникальными сущностями типа `None`, `False` или `True`лучше производить через `is`. Если кто-то из вас пользуется пайчармом, то он мог говорить вам что-то такое

О проверке типа `if cat`, когда `cat` - это различные типы данных

In [42]:
def if_cat(cat):
    if cat:
        return 'yes'
    else:
        return 'no'

In [43]:
if_cat(True), if_cat(False)

('yes', 'no')

In [44]:
if_cat(None)

'no'

In [45]:
if_cat([]), if_cat(''), if_cat({})

('no', 'no', 'no')

In [46]:
if_cat(0), if_cat(1), if_cat(6)

('no', 'yes', 'yes')

In [47]:
if_cat([12, ]), if_cat('hi!'), if_cat({'hi!': 12})

('yes', 'yes', 'yes')

Где тут может быть подвох: дейсвтия на `None` часто требуется прописывать отдельно, и такая проверка может перепутать его с любым другим пустым объектом. Пусть у вас есть функция, в которую передается параметр, отвечающий за кол-во симовлов при выводе. Если он `None`, то вывести надо все

In [48]:
def test_func3(text, line=None):
    # do something
    if not line:
        return text
    else:
        return text[:line]

In [49]:
text = '\nЛиса предложила раку бегать наперегонки. Рак согласился. Лиса побежала, а рак уцепился за лисий хвост. ' + \
'\nЛиса добежала до места. Обернулась лиса, а рак отцепился и говорит: «A я давно тут тебя жду».\n'
text

'\nЛиса предложила раку бегать наперегонки. Рак согласился. Лиса побежала, а рак уцепился за лисий хвост. \nЛиса добежала до места. Обернулась лиса, а рак отцепился и говорит: «A я давно тут тебя жду».\n'

In [50]:
test_func3(text, 10)

'\nЛиса пред'

In [51]:
test_func3(text, None)

'\nЛиса предложила раку бегать наперегонки. Рак согласился. Лиса побежала, а рак уцепился за лисий хвост. \nЛиса добежала до места. Обернулась лиса, а рак отцепился и говорит: «A я давно тут тебя жду».\n'

In [52]:
test_func3(text, 0)

'\nЛиса предложила раку бегать наперегонки. Рак согласился. Лиса побежала, а рак уцепился за лисий хвост. \nЛиса добежала до места. Обернулась лиса, а рак отцепился и говорит: «A я давно тут тебя жду».\n'

Мы же не хотели получать символы в выводе, ввели ноль, а тут вдруг целый текст. Нехорошо :)</br>
Поэтому лучше делать сравнение с `None` явным

## 5. Про скорость и память (немного)

In [55]:
row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])
mx_zero = csr_array((data, (row, col)), shape=(10, 10))
mx_zero.toarray()

array([[1, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 0, 0, 0, 0, 0],
       [4, 5, 6, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [56]:
sys.getsizeof(mx_zero), sys.getsizeof(mx_zero.toarray())

(48, 928)

### Задание
Давайте для каждого текста построим обратный индекс, потом найдем пять самых частотных слов во всем корпусе, а потом (если успеем) посчитаем tf-idf и найдем по три ключевых слова

In [57]:
df = pd.read_csv('data.csv')
df.head(1)

,text
0,"Разнообразный и богатый опыт говорит нам, что ..."


In [58]:
def preprocess_text(text):
    lemmas = []
    for word in word_tokenize(text):
        if word.isalpha():
            word = morph.parse(word.lower())[0]
            lemma = word.normal_form
            if lemma not in stops:
                lemmas.append(lemma)
    return ' '.join(lemmas)

In [59]:
df['clean_text'] = df.text.apply(preprocess_text)
df.head(1)

,text,clean_text
0,"Разнообразный и богатый опыт говорит нам, что ...",разнообразный богатый опыт говорить граница об...


In [60]:
%%timeit
df['clean_text'] = df.text.apply(preprocess_text)
df.head(1)

85.3 ms ± 659 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# пишем обратный индекс в виде словаря

In [80]:
%%timeit
freq_dict = {}
vocab = {}
vocab_list = []
for i, text in enumerate(df.clean_text):
    for word in text.split():
        if word not in vocab:
            vocab[word] = len(vocab)
            vocab_list.append(word)
        num_word = vocab[word]
        
        if num_word not in freq_dict:
            freq_dict[num_word] = {}
        if i not in freq_dict[num_word]:
            freq_dict[num_word][i] = 0
            
        freq_dict[num_word][i] += 1

187 µs ± 2.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [72]:
%%timeit
words_stat = {}
for word in freq_dict:
    words_stat[word] = sum(freq_dict[word].values())
[vocab_list[i[0]] for i in sorted(words_stat.items(), key=lambda x: x[1], 
                                  reverse=True)[:5]]

65.6 µs ± 409 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [92]:
%%timeit
vocab_scp[freq_spm.sum(axis=0).argsort()[:, -1:-6:-1]]

37.1 µs ± 615 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [79]:
%%timeit
cv = CountVectorizer()
freq_spm = cv.fit_transform(df.clean_text)

574 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [83]:
cv = CountVectorizer()
freq_spm = cv.fit_transform(df.clean_text)

In [85]:
freq_spm.toarray()

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 2, 0]])

In [86]:
vocab_scp = cv.get_feature_names_out()